# Train - xgb


리더보드 ACC: 0.3956

In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import torch
# import lightgbm as lgb
from xgboost import XGBClassifier

from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')




def random_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    
    return True
random_seed(seed=42)

True

In [2]:
''' 
train_df = train_label + train_data
test_df = test_label + test_data

all_df = all_label + all_data
'''

dir_path = "/data/ephemeral/home/data"

data_path = "./data.csv"
all_df = pd.read_csv(data_path)

train_df = all_df.loc[all_df["_type"]=="train"].drop(columns=["_type"])
test_df = all_df.loc[all_df["_type"]=="test"].drop(columns=["_type"])
print("data shape: ", all_df.shape)
print("train shape: ", train_df.shape)
print("test shape: ", test_df.shape)

data shape:  (11552, 509)
train shape:  (8760, 508)
test shape:  (2792, 508)


In [3]:
train_df.head()

,ID,target,coinbase_premium_gap,coinbase_premium_index,funding_rates,long_liquidations,long_liquidations_usd,short_liquidations,short_liquidations_usd,open_interest,...,active_count_shift_10,receiver_count_shift_10,sender_count_shift_10,liquidation_diff_shift_10,liquidation_usd_diff_shift_10,volume_diff_shift_10,liquidation_diffg_shift_10,liquidation_usd_diffg_shift_10,volume_diffg_shift_10,buy_sell_volume_ratio_shift_10
0,2023-01-01 00:00:00,2.0,-9.86,-0.059650,0.005049,0.012,197.51610,0.000,0.00000,6.271344e+09,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,2023-01-01 01:00:00,1.0,-8.78,-0.053047,0.005049,0.000,0.00000,0.712,11833.56104,6.288683e+09,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,2023-01-01 02:00:00,1.0,-9.59,-0.057952,0.005049,0.000,0.00000,0.000,0.00000,6.286796e+09,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,2023-01-01 03:00:00,1.0,-9.74,-0.058912,0.005067,0.593,9754.76891,0.000,0.00000,6.284575e+09,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,2023-01-01 04:00:00,2.0,-10.14,-0.061373,0.006210,0.361,5944.43714,0.000,0.00000,6.291582e+09,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [4]:
test_df.head()

,ID,target,coinbase_premium_gap,coinbase_premium_index,funding_rates,long_liquidations,long_liquidations_usd,short_liquidations,short_liquidations_usd,open_interest,...,active_count_shift_10,receiver_count_shift_10,sender_count_shift_10,liquidation_diff_shift_10,liquidation_usd_diff_shift_10,volume_diff_shift_10,liquidation_diffg_shift_10,liquidation_usd_diffg_shift_10,volume_diffg_shift_10,buy_sell_volume_ratio_shift_10
8760,2024-01-01 00:00:00,NaN,-22.57,-0.053137,0.042276,0.661000,27996.83627,17.251477,7.350195e+05,1.034554e+10,...,80380.0,25484.0,58539.0,-3.275801,-1.393753e+05,6.311677e+07,-1.0,-1.0,1.0,1.475138
8761,2024-01-01 01:00:00,NaN,-18.88,-0.044305,0.042578,0.483000,20552.07715,36.152847,1.546929e+06,1.040165e+10,...,66899.0,17439.0,51709.0,1.924824,8.087924e+04,-4.717319e+07,1.0,1.0,-1.0,0.701483
8762,2024-01-01 02:00:00,NaN,-9.78,-0.022968,0.043265,6.724000,285861.94051,1.623000,6.941107e+04,1.039594e+10,...,65812.0,15247.0,52548.0,35.532924,1.493391e+06,-6.240264e+07,1.0,1.0,-1.0,0.740465
8763,2024-01-01 03:00:00,NaN,-5.38,-0.012710,0.043927,20.107080,849377.45559,0.071000,3.003284e+03,1.033749e+10,...,41020.0,15500.0,26760.0,-0.651000,-2.787925e+04,3.778580e+07,-1.0,-1.0,1.0,1.495168
8764,2024-01-01 04:00:00,NaN,-10.22,-0.024104,0.045189,12.654933,533700.00186,1.651000,7.014376e+04,1.039205e+10,...,57263.0,20099.0,38653.0,-5.000800,-2.122320e+05,2.343655e+07,-1.0,-1.0,1.0,1.234126


## Train

In [5]:
y = train_df['target']    # fetures
x = train_df.drop(['ID', 'target'], axis=1)    # target

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

# Train
model = XGBClassifier(
    eval_metric='mlogloss',
    random_state=42
)
model = model.fit(
            x_train, y_train, 
            eval_set=[(x_val, y_val)]
            )

# Val
y_pred = model.predict(x_val)

accuracy = accuracy_score(y_val, y_pred)
print(f"acc: {accuracy}")

(7008, 506) (7008,)
(1752, 506) (1752,)
[0]	validation_0-mlogloss:1.30192
[1]	validation_0-mlogloss:1.24821
[2]	validation_0-mlogloss:1.21166
[3]	validation_0-mlogloss:1.18589
[4]	validation_0-mlogloss:1.17000
[5]	validation_0-mlogloss:1.15877
[6]	validation_0-mlogloss:1.15286
[7]	validation_0-mlogloss:1.14883
[8]	validation_0-mlogloss:1.14665
[9]	validation_0-mlogloss:1.14310
[10]	validation_0-mlogloss:1.14292
[11]	validation_0-mlogloss:1.14097
[12]	validation_0-mlogloss:1.14294
[13]	validation_0-mlogloss:1.14504
[14]	validation_0-mlogloss:1.14827
[15]	validation_0-mlogloss:1.14929
[16]	validation_0-mlogloss:1.15214
[17]	validation_0-mlogloss:1.15507
[18]	validation_0-mlogloss:1.15715
[19]	validation_0-mlogloss:1.16119
[20]	validation_0-mlogloss:1.16139
[21]	validation_0-mlogloss:1.16342
[22]	validation_0-mlogloss:1.16281
[23]	validation_0-mlogloss:1.16580
[24]	validation_0-mlogloss:1.16612
[25]	validation_0-mlogloss:1.16774
[26]	validation_0-mlogloss:1.17159
[27]	validation_0-mloglos

## Feature selection

In [6]:
N = 50

importance = model.feature_importances_
feature_importance = pd.DataFrame({'feature': x_train.columns, 'importance': importance})
feature_importance = feature_importance.sort_values('importance', ascending=False)

selected_features = feature_importance['feature'][:N].tolist()
selected_features

['long_liquidations_usd_rolling_mean_6',
 'sell_volume_rolling_mean_5',
 'volume_diffg_rolling_mean_2',
 'buy_volume_rolling_mean_6',
 'open_interest_rolling_mean_6',
 'liquidation_usd_diff_rolling_mean_2',
 'open_interest_rolling_mean_3',
 'open_interest_rolling_mean_5',
 'open_interest_rolling_mean_7',
 'liquidation_usd_diffg_rolling_std_2',
 'liquidation_usd_diff_rolling_mean_7',
 'sell_volume_rolling_mean_3',
 'sell_volume',
 'liquidation_usd_diffg_rolling_mean_10',
 'sell_volume_rolling_mean_2',
 'open_interest_rolling_mean_9',
 'open_interest_rolling_mean_8',
 'long_liquidations_rolling_std_7',
 'coinbase_premium_index_rolling_mean_9',
 'sell_ratio_rolling_mean_5',
 'short_liquidations_usd_rolling_mean_8',
 'coinbase_premium_index_rolling_mean_4',
 'liquidation_diffg_rolling_mean_2',
 'liquidation_usd_diffg_rolling_std_7',
 'liquidation_usd_diffg_rolling_std_8',
 'sell_volume_rolling_mean_8',
 'buy_volume',
 'coinbase_premium_index_rolling_mean_2',
 'open_interest_rolling_mean_10

In [7]:
# selected features
train_df = train_df[['ID', 'target']+selected_features]

y = train_df['target']    # fetures
x = train_df.drop(['ID', 'target'], axis=1)    # target

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

# Train
model = XGBClassifier(
    eval_metric='mlogloss',
    random_state=42
)
model = model.fit(
            x_train, y_train, 
            eval_set=[(x_val, y_val)]
            )

# Val
y_pred = model.predict(x_val)

accuracy = accuracy_score(y_val, y_pred)
print(f"acc: {accuracy}")

(7008, 50) (7008,)
(1752, 50) (1752,)
[0]	validation_0-mlogloss:1.29939
[1]	validation_0-mlogloss:1.24500
[2]	validation_0-mlogloss:1.20843
[3]	validation_0-mlogloss:1.18553
[4]	validation_0-mlogloss:1.17019
[5]	validation_0-mlogloss:1.15747
[6]	validation_0-mlogloss:1.15012
[7]	validation_0-mlogloss:1.14687
[8]	validation_0-mlogloss:1.14561
[9]	validation_0-mlogloss:1.14412
[10]	validation_0-mlogloss:1.14355
[11]	validation_0-mlogloss:1.14502
[12]	validation_0-mlogloss:1.14672
[13]	validation_0-mlogloss:1.14833
[14]	validation_0-mlogloss:1.14974
[15]	validation_0-mlogloss:1.15015
[16]	validation_0-mlogloss:1.15233
[17]	validation_0-mlogloss:1.15398
[18]	validation_0-mlogloss:1.15542
[19]	validation_0-mlogloss:1.15745
[20]	validation_0-mlogloss:1.15997
[21]	validation_0-mlogloss:1.16159
[22]	validation_0-mlogloss:1.16318
[23]	validation_0-mlogloss:1.16548
[24]	validation_0-mlogloss:1.16822
[25]	validation_0-mlogloss:1.16964
[26]	validation_0-mlogloss:1.16979
[27]	validation_0-mlogloss:

## Predict

In [8]:
# selected features
test_df = test_df[['ID', 'target']+selected_features]

test_data = test_df.drop(["target", "ID"], axis=1)

y_test_pred = model.predict(test_data)

submission_df = pd.read_csv(f"{dir_path}/test.csv")
submission_df = submission_df.assign(target=y_test_pred)
submission_df.to_csv("output.csv", index=False)

print(submission_df.target.value_counts())
submission_df

target
2    1634
1     926
0     181
3      51
Name: count, dtype: int64


,ID,target
0,2024-01-01 00:00:00,2
1,2024-01-01 01:00:00,2
2,2024-01-01 02:00:00,1
3,2024-01-01 03:00:00,2
4,2024-01-01 04:00:00,2
...,...,...
2787,2024-04-26 03:00:00,2
2788,2024-04-26 04:00:00,1
2789,2024-04-26 05:00:00,2
2790,2024-04-26 06:00:00,2
